# Four tank system example - Fault Information System, row resoning

In [1]:
import pandas as pd
from isolability import pairwise_relationships as pr
from isolability import martices_and_indices as ind
from four_tanks.es import ess

#### Fault Information System

In [2]:
fis_df = pd.read_csv('four_tanks/fis.csv', sep=';', index_col=0, dtype=str)
fis_df

,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10
s_1,-1,-1,-1,-1,+1,+1,+1,"-1,+1","-1,+1",0
s_2,-1,-1,-1,-1,-1,-1,-1,"-1,+1",0,"-1,+1"
s_3,0,-1,-1,-1,-1,-1,-1,0,"-1,+1","-1,+1"


In [3]:
fis_df = fis_df.map(lambda x: set(a.strip() for a in x.split(',')))
fis_df

,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10
s_1,{-1},{-1},{-1},{-1},{+1},{+1},{+1},"{+1, -1}","{+1, -1}",{0}
s_2,{-1},{-1},{-1},{-1},{-1},{-1},{-1},"{+1, -1}",{0},"{+1, -1}"
s_3,{0},{-1},{-1},{-1},{-1},{-1},{-1},{0},"{+1, -1}","{+1, -1}"


#### Exploration of pairwise isolabity relationships
* Function arguments are fault **indices** starting from 0, so pr.RS_R_FIS(fis_df, 1, 2) checks faults $f_2$ and $f_3$
* Avaliable relations:
    - RN_R_FIS - non-isolable
    - RUN_R_FIS - unilaterally non-isolable
    - RUW_R_FIS - unilaterally (weakly) isolable
    - RW_R_FIS - bilaterally (weakly) isolable
    - RUC_R_FIS - unilaterally conditionally isolable
    - RC_R_FIS - bilaterally conditionally isolable
    - RUS_R_FIS - unilaterally strongly isolable
    - RS_R_FIS - bilaterally strongly isolable

In [5]:
pr.RS_R_FIS(fis_df, 7, 8)

False

#### EIM matrix

In [6]:
eim = ind.EIM_R_FIS(fis_df)
eim

array([[0. , 0. , 0. , 0. , 1. , 1. , 1. , 0. , 1. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 0. , 0. , 0. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 0. , 0. , 0. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 0. , 0. , 0. , 1. , 1. , 1. ],
       [0.4, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 1. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 1. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 1. , 0. ]])

#### $C\Delta$ coefficient

In [7]:
c_delta = ind.C_Delta(eim)
print(f"{c_delta:.3f}")

0.616


#### $C^*\Delta$ coefficient

In [8]:
c_star_delta = ind.C_star_Delta(eim)
print(f"{c_star_delta:.3f}")

0.616


#### IMES matrix

In [9]:
imes = ind.IMES(ess)
imes

array([[0., 0., 1., 1., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 1., 0., 0., 0.],
       [1., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [11]:
comp_weights = pd.read_csv('conf/IMES_weights_FIS_R.csv', sep=';', index_col=0)
comp_weights

,RW_ES,RJW_ES,RN_ES
RS_Rn_FIS,2.0,2.0,2.0
RUS_Rn_FIS,2.0,2.0,1.5
RW_R_FIS,2.0,1.5,1.0
RUW_R_FIS,2.0,1.5,1.0
RC_R_FIS,1.7,1.2,0.7
RUC_R_FIS,1.4,0.9,0.4
RUN_R_FIS,1.0,0.5,0.0
RN_R_FIS,1.0,0.5,0.0


In [12]:
eim_star = ind.EIM_star_R_FIS(fis_df, ess, comp_weights)
eim_star

array([[0. , 0. , 1. , 1. , 1. , 1. , 2. , 0. , 1. , 1. ],
       [1. , 0. , 1. , 1. , 1. , 1. , 2. , 1. , 1. , 1. ],
       [2. , 1. , 0. , 0. , 2. , 1. , 1. , 1. , 1. , 1. ],
       [2. , 1. , 0. , 0. , 2. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 2. , 2. , 0. , 0. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 0. , 0. , 0. , 1. , 1. , 1. ],
       [2. , 2. , 1. , 1. , 1. , 0. , 0. , 1. , 1. , 1. ],
       [0.4, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 1. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 1. ],
       [1. , 0. , 0. , 0. , 0. , 0. , 0. , 1. , 1. , 0. ]])

In [13]:
c_delta = ind.C_Delta(eim_star)
print(f"{c_delta:.3f}")

0.816


In [14]:
c_star_delta = ind.C_star_Delta(eim_star)
print(f"{c_star_delta:.3f}")

0.704
